In [ ]:
def get_deltas(case_data): #get dayt over day % change ... doing to find r0 modeling function.... because its not quite logistic
    i = 0
    ret = [0] #to be same size as data in
    while i < len(case_data)-1:
        ret.append( (case_data[i+1] - case_data[i])/case_data[i])
        i= i+1
    
    return ret

In [ ]:
p_change_real = get_deltas(t_cases)

In [ ]:
#r0 function analysis... https://scipy-cookbook.readthedocs.io/items/robust_regression.html
from scipy.optimize import least_squares

def generate_data(t, A, sigma, omega, noise=0, n_outliers=0, random_state=0):
    y = A * np.exp(-sigma * t) * np.sin(omega * t)
    rnd = np.random.RandomState(random_state)
    error = noise * rnd.randn(t.size)
    outliers = rnd.randint(0, t.size, n_outliers)
    error[outliers] *= 35
    return y + error


#for residuals
def fun(x, t, y):
    return x[0] * np.exp(-x[1] * t) * np.sin(x[2] * t) - y

x0 = np.ones(3)
leng= len(p_change_real)
t_train = np.linspace(0,leng,leng)
y_train= p_change_real

res_robust = least_squares(fun, x0, loss='soft_l1', f_scale=0.1, args=(t_train, y_train))

y_robust = generate_data(t_train, *res_robust.x)

fig, ax = plt.subplots()
ax.plot(t_days, p_change_real, 'o', alpha=0.7, label='daily perecent change')
ax.plot(t_days, y_robust, 'r', label='robust lsq')
# rotate and align the tick labels so they look better
fig.autofmt_xdate()
ax.xaxis.set_major_locator(WeekdayLocator(byweekday=MO))